# Titanic Survival Prediction using Logistic Regression
(Make sure to download the datasets from [Kaggle](https://www.kaggle.com/competitions/titanic) and store them in "datasets" folder)

Objective: Predict if a passanger survived the sinking of the Titanic or not.

In [1]:
import pandas as pd
from sklearn.impute import KNNImputer

In [2]:
# Import all datasets
train = pd.read_csv('datasets/train_data.csv')
test = pd.read_csv('datasets/test_data.csv')
test_survival = pd.read_csv('datasets/test_survival.csv')

## Display all datasets

In [3]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [5]:
test_survival.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


## Data Cleaning

### Handling missing data

In [6]:
# Check missing data in the datasets
print("Training Dataset\n", train.isnull().sum())
print("Testing Dataset\n", test.isnull().sum())

Training Dataset
 PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64
Testing Dataset
 PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64


In [7]:
# Drop the 'Cabin' column as it has too many missing values
train.drop(columns=['Cabin'], inplace=True)
test.drop(columns=['Cabin'], inplace=True)

# Fill missing values in 'Age' column using K-Nearest Neighbors imputation
imputer = KNNImputer(n_neighbors=5)
train[['Age']] = imputer.fit_transform(train[['Age']])
test[['Age']] = imputer.transform(test[['Age']])

# Fill missing values in 'Embarked' column with the mode
train['Embarked'] = train['Embarked'].fillna(train['Embarked'].mode()[0])

# Fill missing values in 'Fare' column with the mean
test['Fare'] = test['Fare'].fillna(test['Fare'].mean())

### Correcting data types

In [8]:
# Check data types of all columns
print("Training Dataset\n", train.dtypes)
print("Testing Dataset\n", test.dtypes)
print("Testing Survival Dataset\n", test_survival.dtypes)

Training Dataset
 PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Embarked        object
dtype: object
Testing Dataset
 PassengerId      int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Embarked        object
dtype: object
Testing Survival Dataset
 PassengerId    int64
Survived       int64
dtype: object


In [9]:
# Convert "Sex" and "Embarked" columns to categorical data type
train["Sex"] = train["Sex"].astype('category')
test["Sex"] = test["Sex"].astype('category')
train["Embarked"] = train["Embarked"].astype('category')
test["Embarked"] = test["Embarked"].astype('category')

### Removing Duplicates

In [10]:
# Check for duplicate rows
print("Training Dataset\n", train.duplicated().sum())
print("Testing Dataset\n", test.duplicated().sum())
print("Testing Survival Dataset\n", test_survival.duplicated().sum())

Training Dataset
 0
Testing Dataset
 0
Testing Survival Dataset
 0


## Modelling (using Logistic Regression)

In [11]:
# Import library
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [12]:
# Merge testing dataset with the testing survival dataset
test = test.merge(test_survival, on='PassengerId')
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Embarked,Survived
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,Q,0
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,S,1
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,Q,0
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,S,0
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,S,1


In [13]:
# We already have test and train datasets separated so we can directly use them
X_train = train.drop(columns=['PassengerId', 'Survived', 'Name', 'Ticket'])
y_train = train['Survived']
X_test = test.drop(columns=['PassengerId', 'Survived', 'Name', 'Ticket'])
y_test = test['Survived']

# Convert categorical data to numerical data
X_train = pd.get_dummies(X_train)
X_test = pd.get_dummies(X_test)

In [14]:
# Train the model with increased max_iter
model = LogisticRegression(max_iter=500)
model.fit(X_train, y_train)

# Make predictions
predictions = model.predict(X_test)

# Evaluate the model
print("Classification Report\n", classification_report(y_test, predictions))
print("Confusion Matrix\n", confusion_matrix(y_test, predictions))
print("Accuracy\n", accuracy_score(y_test, predictions))

Classification Report
               precision    recall  f1-score   support

           0       0.96      0.95      0.95       266
           1       0.92      0.93      0.92       152

    accuracy                           0.94       418
   macro avg       0.94      0.94      0.94       418
weighted avg       0.94      0.94      0.94       418

Confusion Matrix
 [[253  13]
 [ 11 141]]
Accuracy
 0.9425837320574163


Recommendation: Might as well find better algorithm